In [ ]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Model Storage Path**

In [ ]:
model_path = '/content/drive/MyDrive/UIElements/mini/mini/model/mask.h5'


**Load Dataset**

In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/mini/train/train"
TEST_DIR = "/content/drive/MyDrive/mini/test/test"


**Split Data**

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.1,  # 10% for validation
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    subset='validation',
    shuffle=False,
    seed=42
)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)


**Model**

In [ ]:
model = Sequential([
    Conv2D(256, (3,3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn import metrics


checkpoint = ModelCheckpoint(model_path,monitor='val_loss',verbose=1,save_best_only=True,mode='auto')

# early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)


history = model.fit(train_generator,
                              epochs=100,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

test_generator.reset()
predictions = np.argmax(model.predict(test_generator), axis=-1)
true_labels = test_generator.classes

# Metrics
accuracy = accuracy_score(true_labels, predictions)
print("Face Recognition Accuracy:", accuracy)

precision_scores = precision_score(true_labels, predictions, average=None)
print("Precision per class:", precision_scores)
print("Mean Precision:", precision_scores.mean())

recall_scores = recall_score(true_labels, predictions, average=None)
print("Recall per class:", recall_scores)
print("Mean Recall:", recall_scores.mean())

f1_scores = f1_score(true_labels, predictions, average=None)
print("F1 Score per class:", f1_scores)
print("Mean F1 Score:", f1_scores.mean())


print("Classification Report:")
print(classification_report(true_labels, predictions, target_names=test_generator.class_indices.keys()))


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.figure(figsize=(12, 5))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


Test Accuracy and Confusion matrix

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Load the best model
model = load_model('/content/drive/MyDrive/UIElements/mini/mini/model/finalmask1.h5')

# Predict on test data
test_generator.reset()
predictions = np.argmax(model.predict(test_generator), axis=-1)

# True labels
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
print("Test Accuracy:", accuracy)

precision = precision_score(true_labels, predictions, average=None)
recall = recall_score(true_labels, predictions, average=None)
f1 = f1_score(true_labels, predictions, average=None)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(true_labels, predictions, target_names=class_labels))

# Confusion matrix
conf_matrix = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
